## PT complex tensor examples

In [1]:
import numpy as np
import torch
from organics.utils.complex_tensors import ComplexTensor
import tensorflow as tf
tf.enable_eager_execution()

---   
Creation

In [2]:
# numpy complex tensor
np_c = np.asarray([[1+3j, 1+3j, 1+3j], [2+4j, 2+4j, 2+4j]]).astype(np.complex64)
np_c

array([[1.+3.j, 1.+3.j, 1.+3.j],
       [2.+4.j, 2.+4.j, 2.+4.j]], dtype=complex64)

In [3]:
# torch equivalent
pt_c = ComplexTensor([[1, 1, 1], [2,2,2], [3,3,3], [4,4,4]])
pt_c

tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.],
        [4., 4., 4.]])

In [4]:
# verify reals match
print(np_c.real)
print(pt_c.real)

[[1. 1. 1.]
 [2. 2. 2.]]
tensor([[1., 1., 1.],
        [2., 2., 2.]])


In [5]:
# verify imag match
print(np_c.imag)
print(pt_c.imag)

[[3. 3. 3.]
 [4. 4. 4.]]
tensor([[3., 3., 3.],
        [4., 4., 4.]])


---   
Verify complex addition

In [6]:
np_c + (3+2j)

array([[4.+5.j, 4.+5.j, 4.+5.j],
       [5.+6.j, 5.+6.j, 5.+6.j]], dtype=complex64)

In [7]:
pt_c + (3 + 2j)

tensor([[4., 4., 4.],
        [5., 5., 5.],
        [5., 5., 5.],
        [6., 6., 6.]])

--- 
verify abs

In [8]:
np.abs(np_c)

array([[3.1622777, 3.1622777, 3.1622777],
       [4.472136 , 4.472136 , 4.472136 ]], dtype=float32)

In [9]:
pt_c.abs()

tensor([[3.1623, 3.1623, 3.1623],
        [4.4721, 4.4721, 4.4721]])

--- 
verify complex vs real matrix multiply

In [10]:
np_x = np.asarray([[3, 3], [4, 4], [2, 2]])
pt_x = torch.Tensor([[3, 3], [4, 4], [2, 2]])

print(np_x)
print(pt_x)

[[3 3]
 [4 4]
 [2 2]]
tensor([[3., 3.],
        [4., 4.],
        [2., 2.]])


In [11]:
np_mm_out = np.matmul(np_c, np_x)
np_mm_out

array([[ 9.+27.j,  9.+27.j],
       [18.+36.j, 18.+36.j]])

In [12]:
pt_mm_out = pt_c.mm(pt_x)
pt_mm_out

tensor([[ 9.,  9.],
        [18., 18.],
        [27., 27.],
        [36., 36.]])

In [13]:
# verify reals
print(np_mm_out.real)
print(pt_mm_out.real)

[[ 9.  9.]
 [18. 18.]]
tensor([[ 9.,  9.],
        [18., 18.]])


In [14]:
# verify imags
print(np_mm_out.imag)
print(pt_mm_out.imag)

[[27. 27.]
 [36. 36.]]
tensor([[27., 27.],
        [36., 36.]])


--- 
verify transpose

In [15]:
np_c.T

array([[1.+3.j, 2.+4.j],
       [1.+3.j, 2.+4.j],
       [1.+3.j, 2.+4.j]], dtype=complex64)

In [16]:
pt_c.t()

tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [3., 4.],
        [3., 4.],
        [3., 4.]])

--- 
## See if gradients still work 


In [17]:
pt_c2 = ComplexTensor([[2, 2, 2], [1,1,1], [4,4,4], [3,3,3]])
pt_c2.requires_grad = True

In [18]:
out = pt_c2 + 4
out = out.mm(pt_c.t())
print(out)

tensor([[-18., -12.],
        [-12.,  -6.],
        [ 66.,  96.],
        [ 54.,  78.]], grad_fn=<CatBackward>)


In [19]:
y = out.sum()
print(y)

tensor(246., grad_fn=<SumBackward0>)


In [20]:
y.backward()

In [21]:
pt_c2.grad

tensor([[10., 10., 10.],
        [10., 10., 10.],
        [-4., -4., -4.],
        [-4., -4., -4.]])

---  
### Use Tensorflow to compute grads


In [2]:
tf_c = tf.constant([[1+3j, 1+3j, 1+3j], [2+4j, 2+4j, 2+4j]], dtype=tf.complex64)
tf_c2 = tf.constant([[2+4j, 2+4j, 2+4j], [1+3j,1+3j,1+3j]], dtype=tf.complex64)

In [3]:
with tf.GradientTape() as t:
    t.watch(tf_c2)
    tf_out = tf_c2 + 4
    tf_out = tf.matmul(tf_out, tf.transpose(tf_c, perm=[1,0]))
    print(tf_out)
    
    tf_y = tf.reduce_sum(tf_out)
    print(tf_y)

tf.Tensor(
[[-18.+66.j -12.+96.j]
 [-12.+54.j  -6.+78.j]], shape=(2, 2), dtype=complex64)
tf.Tensor((-48+294j), shape=(), dtype=complex64)


In [4]:
dy_dc2 = t.gradient(tf_y, tf_c2)

In [5]:
dy_dc2

<tf.Tensor: id=27, shape=(2, 3), dtype=complex64, numpy=
array([[3.-7.j, 3.-7.j, 3.-7.j],
       [3.-7.j, 3.-7.j, 3.-7.j]], dtype=complex64)>